# Imports

In [1]:
import mysql.connector as mysqlConnector
import pandas as pd 
import numpy as np

# Connection à la bdd

In [2]:
conn = mysqlConnector.connect(host='Localhost',user='root',passwd='*****', database='dataengineer')
if conn:
    print("Connection Successful!")
else:
    print("Connection Failed!")
cur = conn.cursor()
cur.execute("SHOW DATABASES")
for row in cur:
    print(row)

Connection Successful!
('dataengineer',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testwildcode',)


In [3]:
cur.execute("SHOW TABLES")
for row in cur:
    print(row)

('actor',)
('address',)
('address_test',)
('category',)
('customer',)
('film',)
('film_actor',)
('film_category',)
('film_text',)
('inventory',)
('language',)
('payment',)
('rental',)
('staff',)
('store',)


In [4]:
cur.execute("SELECT * FROM address_test LIMIT 2")
for row in cur:
    print(row)

(0, 1, '318 CHE DE ROUMAGOUA', 'LA CIOTAT', '13600', 43.197642, 5.6078904)
(1, 2, '19 RUE DES DAMES', 'SAINTE SAVINE', '10300', 48.2909051, 4.0457894)


# Modification de la table address avec ajout des 2 colonnes latitude et longitude

In [ ]:
cur.execute("ALTER TABLE address ADD COLUMN latitude DECIMAL(11,7),ADD COLUMN longitude DECIMAL(11,7);")

In [12]:
# Set SQL query as a comment
sql_query = ''' SELECT * FROM address '''

# Use pandas to pass sql query using connection form SQLite3
df = pd.read_sql(sql_query, conn)

# Show the resulting DataFrame
df

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.197642,5.607890
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,5100,NaN,NaN
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,48.953836,2.292919


## Ajout des données correspondantes

In [9]:
cur.execute("UPDATE address ad1 INNER JOIN address_test ad2 ON ad1.address_id = ad2.address_id SET ad1.latitude = ad2.latitude;")

In [11]:
cur.execute("UPDATE address ad1 INNER JOIN address_test ad2 ON ad1.address_id = ad2.address_id SET ad1.longitude = ad2.longitude;")

## Suppression de la table temporaire address_test

In [18]:
cur.execute("DROP TABLE address_test;")

# Overview des autres tables pour la requête SQL 

In [13]:
def sql_to_df(sql_query):
    df = pd.read_sql(sql_query, conn)
    return df

In [14]:
query = ''' SELECT *
            FROM customer LIMIT 2; '''

sql_to_df(query).head()

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [15]:
query = ''' SELECT *
            FROM rental LIMIT 2; '''
sql_to_df(query).head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


## Requête SQL 
Nom du client, Prénom, Nombre de locations, Adresses postales, latitude et longitude. Il a uniquement besoin du client qui a fait le plus de locations de toute la base, afin de le féliciter.

In [20]:
query = ''' SELECT last_name, first_name, COUNT(rental_id) AS nb_loc, address, city, postal_code, latitude, longitude 
            FROM customer c, rental r, address a 
            WHERE c.customer_id = r.customer_id AND c.address_id = a.address_id
            GROUP BY r.customer_id 
            ORDER BY nb_loc DESC
            LIMIT 1; '''

sql_to_df(query).head()

,last_name,first_name,nb_loc,address,city,postal_code,latitude,longitude
0,HUNT,ELEANOR,46,17 IMP DES JARDINS,VALLEIRY,74520,None,None
